# Notebook 1: Data Foundation for Marketing Mix Modeling
## Understanding and Preparing Your Data

**Learning Objectives:**
- Understand the three key data sources for MMM
- Learn how to aggregate data hierarchically
- Master time-series data alignment
- Create a unified dataset for modeling

---

## Why Data Foundation Matters

Marketing Mix Modeling requires integrating multiple data sources at different granularities. The quality of your data pipeline directly impacts model accuracy. We'll work with:
1. **Sales Data** (daily transactions)
2. **Marketing Investment** (monthly budgets)
3. **Brand Health Metrics** (NPS scores)

Each source has different frequencies and hierarchies that we must carefully align.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("Libraries loaded successfully!")

## Step 1: Loading the Three Data Sources

We have three critical data files that form the foundation of our MMM:
- **firstfile.csv**: Daily sales transactions with product hierarchy
- **MediaInvestment.csv**: Monthly marketing channel investments
- **MonthlyNPSscore.csv**: Monthly Net Promoter Score (brand health indicator)

Let's load and explore each one:

In [ ]:
def load_and_explore_data():
    '''
    Load all three data sources and display their characteristics.
    This function helps us understand what we're working with.
    '''

    # Load sales data - our primary response variable source
    sales_df = pd.read_csv('data/firstfile.csv')
    print("SALES DATA OVERVIEW")
    print("=" * 50)
    print(f"Shape: {sales_df.shape[0]:,} rows  {sales_df.shape[1]} columns")
    print(f"Date range: {sales_df['Date'].min()} to {sales_df['Date'].max()}")
    print(f"\nColumns: {', '.join(sales_df.columns.tolist())}")
    print(f"\nProduct Categories: {sales_df['product_category'].nunique()}")
    print(f"Product Subcategories: {sales_df['product_subcategory'].nunique()}")

    # Load marketing investment data - our key predictors
    marketing_df = pd.read_csv('data/MediaInvestment.csv')
    print("\n" + "=" * 50)
    print("MARKETING INVESTMENT DATA OVERVIEW")
    print("=" * 50)
    print(f"Shape: {marketing_df.shape[0]:,} rows  {marketing_df.shape[1]} columns")

    # Clean column names (remove any leading/trailing spaces)
    marketing_df.columns = marketing_df.columns.str.strip()

    # Identify marketing channels
    marketing_channels = [col for col in marketing_df.columns
                         if col not in ['Date', 'Total Investment']]
    print(f"Marketing Channels: {', '.join(marketing_channels)}")
    print(f"Total Channels: {len(marketing_channels)}")

    # Load NPS data - brand health indicator
    nps_df = pd.read_csv('data/MonthlyNPSscore.csv')
    print("\n" + "=" * 50)
    print("NPS (BRAND HEALTH) DATA OVERVIEW")
    print("=" * 50)
    print(f"Shape: {nps_df.shape[0]:,} rows  {nps_df.shape[1]} columns")
    print(f"NPS Range: {nps_df['NPS'].min():.1f} to {nps_df['NPS'].max():.1f}")

    return sales_df, marketing_df, nps_df, marketing_channels

# Load all data
sales_df, marketing_df, nps_df, marketing_channels = load_and_explore_data()

# Display sample data
print("\nSample Sales Data:")
display(sales_df.head())

## Step 2: Data Quality Assessment

Before merging, let's check for data quality issues:

In [ ]:
def check_data_quality(sales_df, marketing_df, nps_df):
    '''
    Perform comprehensive data quality checks.
    Good data quality is essential for reliable MMM results.
    '''

    print("DATA QUALITY REPORT")
    print("=" * 60)

    # Check for missing values
    print("\n1. MISSING VALUES CHECK:")
    print("-" * 40)

    print("Sales Data:")
    sales_missing = sales_df.isnull().sum()
    if sales_missing.sum() == 0:
        print("  [OK] No missing values")
    else:
        print(f"   Missing values found:")
        print(sales_missing[sales_missing > 0])

    print("\nMarketing Data:")
    marketing_missing = marketing_df.isnull().sum()
    if marketing_missing.sum() == 0:
        print("  [OK] No missing values")
    else:
        print(f"   Missing values found:")
        print(marketing_missing[marketing_missing > 0])

    # Check date formats
    print("\n2. DATE CONSISTENCY CHECK:")
    print("-" * 40)

    sales_df['Date'] = pd.to_datetime(sales_df['Date'])
    marketing_df['Date'] = pd.to_datetime(marketing_df['Date'])
    nps_df['Date'] = pd.to_datetime(nps_df['Date'])

    print(f"Sales: {sales_df['Date'].min()} to {sales_df['Date'].max()}")
    print(f"Marketing: {marketing_df['Date'].min()} to {marketing_df['Date'].max()}")
    print(f"NPS: {nps_df['Date'].min()} to {nps_df['Date'].max()}")

    # Check for negative values
    print("\n3. NEGATIVE VALUES CHECK:")
    print("-" * 40)

    if (sales_df['GMV'] < 0).any():
        print("   Negative GMV values found!")
    else:
        print("  [OK] All GMV values are non-negative")

    if (sales_df['Units'] < 0).any():
        print("   Negative Units found!")
    else:
        print("  [OK] All Units are non-negative")

    # Check marketing spend
    for channel in marketing_channels:
        if channel in marketing_df.columns:
            if (marketing_df[channel] < 0).any():
                print(f"   Negative spend in {channel}")

    return sales_df, marketing_df, nps_df

# Run quality checks
sales_df, marketing_df, nps_df = check_data_quality(sales_df, marketing_df, nps_df)

## Step 3: Hierarchical Data Aggregation

A key insight for MMM is that marketing effects can vary by product hierarchy. We'll aggregate sales data by category and subcategory to capture these patterns:

In [ ]:
def create_hierarchical_aggregation(sales_df):
    '''
    Aggregate sales data by product hierarchy.
    This captures how different product groups respond to marketing.

    Why this matters:
    - Premium products may respond differently to promotions
    - Category-level effects can be stronger than individual SKUs
    - Reduces noise while preserving important patterns
    '''

    print("Creating Hierarchical Aggregation...")
    print("=" * 60)

    # Group by date and product hierarchy
    hierarchy_agg = sales_df.groupby(
        ['Date', 'product_category', 'product_subcategory']
    ).agg({
        'GMV': 'sum',           # Total revenue
        'Units': 'sum',         # Total units sold
        'Avg_MRP': 'mean',      # Average list price
        'Avg_Price': 'mean'     # Average selling price
    }).reset_index()

    print(f"\nAggregation Results:")
    print(f"  Original records: {len(sales_df):,}")
    print(f"  Aggregated records: {len(hierarchy_agg):,}")
    print(f"  Compression ratio: {len(sales_df)/len(hierarchy_agg):.1f}x")

    # Calculate additional metrics
    hierarchy_agg['Avg_Discount'] = hierarchy_agg['Avg_MRP'] - hierarchy_agg['Avg_Price']
    hierarchy_agg['Discount_Pct'] = (hierarchy_agg['Avg_Discount'] /
                                     (hierarchy_agg['Avg_MRP'] + 0.01)) * 100

    # Show hierarchy structure
    print(f"\nHierarchy Structure:")
    print(f"  Categories: {hierarchy_agg['product_category'].nunique()}")
    print(f"  Subcategories: {hierarchy_agg['product_subcategory'].nunique()}")
    print(f"  Date range: {hierarchy_agg['Date'].nunique()} unique days")

    return hierarchy_agg

# Create hierarchical aggregation
hierarchy_data = create_hierarchical_aggregation(sales_df)

# Visualize hierarchy
plt.figure(figsize=(12, 5))

# Plot 1: Sales by category
plt.subplot(1, 2, 1)
category_sales = hierarchy_data.groupby('product_category')['GMV'].sum().sort_values()
category_sales.plot(kind='barh')
plt.title('Total Sales by Product Category')
plt.xlabel('GMV (Total)')
plt.ylabel('Category')

# Plot 2: Number of subcategories per category
plt.subplot(1, 2, 2)
subcat_count = hierarchy_data.groupby('product_category')['product_subcategory'].nunique()
subcat_count.plot(kind='bar')
plt.title('Subcategories per Category')
plt.xlabel('Category')
plt.ylabel('Number of Subcategories')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Step 4: Time-Series Alignment

Marketing data is monthly but sales are daily. We need to align these different frequencies intelligently:

In [ ]:
def align_time_frequencies(hierarchy_data, marketing_df, nps_df):
    '''
    Align different time frequencies across data sources.

    Strategy:
    1. Keep sales at daily granularity (most detailed)
    2. Interpolate monthly marketing to daily (smooth transitions)
    3. Forward-fill NPS scores (brand perception changes slowly)

    This preserves maximum information while ensuring alignment.
    '''

    print("Aligning Time Frequencies...")
    print("=" * 60)

    # Create daily date range
    date_range = pd.date_range(
        start=hierarchy_data['Date'].min(),
        end=hierarchy_data['Date'].max(),
        freq='D'
    )

    print(f"Daily date range: {len(date_range)} days")
    print(f"From {date_range[0]} to {date_range[-1]}")

    # Expand marketing data to daily
    print("\nExpanding Marketing Data (Monthly -> Daily):")

    # Create daily scaffold
    marketing_daily = pd.DataFrame({'Date': date_range})
    marketing_daily['YearMonth'] = marketing_daily['Date'].dt.to_period('M')
    marketing_df['YearMonth'] = marketing_df['Date'].dt.to_period('M')

    # Merge monthly data
    marketing_daily = marketing_daily.merge(
        marketing_df.drop('Date', axis=1),
        on='YearMonth',
        how='left'
    )

    # Interpolate marketing spend (linear interpolation for smooth transitions)
    for channel in marketing_channels:
        if channel in marketing_daily.columns:
            # First forward fill to handle initial NaN
            marketing_daily[channel] = marketing_daily[channel].fillna(method='ffill')
            # Then interpolate for smooth transitions
            marketing_daily[channel] = marketing_daily[channel].interpolate(method='linear')
            # Finally fill any remaining NaN with 0
            marketing_daily[channel] = marketing_daily[channel].fillna(0)

    print(f"  [OK] Interpolated {len(marketing_channels)} channels to daily")

    # Expand NPS to daily
    print("\nExpanding NPS Data (Monthly -> Daily):")
    nps_daily = pd.DataFrame({'Date': date_range})
    nps_daily['YearMonth'] = nps_daily['Date'].dt.to_period('M')
    nps_df['YearMonth'] = nps_df['Date'].dt.to_period('M')

    nps_daily = nps_daily.merge(
        nps_df[['YearMonth', 'NPS']],
        on='YearMonth',
        how='left'
    )

    # Forward fill NPS (assumption: brand perception changes slowly)
    nps_daily['NPS'] = nps_daily['NPS'].fillna(method='ffill')
    nps_daily['NPS'] = nps_daily['NPS'].fillna(nps_daily['NPS'].mean())

    print(f"  [OK] Expanded NPS to daily using forward-fill")

    return marketing_daily, nps_daily

# Align time frequencies
marketing_daily, nps_daily = align_time_frequencies(hierarchy_data, marketing_df, nps_df)

# Visualize the alignment
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Plot 1: Original monthly marketing
axes[0].plot(marketing_df['Date'], marketing_df['TV'], 'o-', label='Monthly TV Spend')
axes[0].set_title('Original Monthly Marketing Data')
axes[0].set_ylabel('Spend')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Interpolated daily marketing
axes[1].plot(marketing_daily['Date'], marketing_daily['TV'], '-', label='Daily TV Spend (Interpolated)', alpha=0.7)
axes[1].set_title('Interpolated Daily Marketing Data')
axes[1].set_ylabel('Spend')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: NPS daily
axes[2].plot(nps_daily['Date'], nps_daily['NPS'], '-', label='Daily NPS (Forward-filled)', color='green')
axes[2].set_title('Daily NPS Scores')
axes[2].set_ylabel('NPS')
axes[2].set_xlabel('Date')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 5: Creating the Unified Dataset

Now we merge all aligned data sources into a single comprehensive dataset:

In [ ]:
def create_unified_dataset(hierarchy_data, marketing_daily, nps_daily):
    '''
    Create the final unified dataset for modeling.
    This is the foundation for all subsequent analysis.
    '''

    print("Creating Unified Dataset...")
    print("=" * 60)

    # Start with hierarchy data
    unified_data = hierarchy_data.copy()

    # Merge marketing data
    print("Merging marketing data...")
    marketing_cols = ['Date'] + marketing_channels
    unified_data = unified_data.merge(
        marketing_daily[marketing_cols],
        on='Date',
        how='left'
    )

    # Merge NPS data
    print("Merging NPS data...")
    unified_data = unified_data.merge(
        nps_daily[['Date', 'NPS']],
        on='Date',
        how='left'
    )

    # Fill any remaining NaN values
    for col in marketing_channels:
        if col in unified_data.columns:
            unified_data[col] = unified_data[col].fillna(0)

    unified_data['NPS'] = unified_data['NPS'].fillna(unified_data['NPS'].mean())

    # Add time features for seasonality
    unified_data['Year'] = unified_data['Date'].dt.year
    unified_data['Month'] = unified_data['Date'].dt.month
    unified_data['Quarter'] = unified_data['Date'].dt.quarter
    unified_data['DayOfWeek'] = unified_data['Date'].dt.dayofweek
    unified_data['WeekOfYear'] = unified_data['Date'].dt.isocalendar().week.astype(int)

    print(f"\nFinal Dataset Shape: {unified_data.shape}")
    print(f"  Rows: {unified_data.shape[0]:,}")
    print(f"  Columns: {unified_data.shape[1]}")
    print(f"\nFeatures included:")
    print(f"  - Sales metrics: GMV, Units, Prices")
    print(f"  - Marketing channels: {len(marketing_channels)}")
    print(f"  - Brand health: NPS")
    print(f"  - Time features: Year, Month, Quarter, etc.")
    print(f"  - Product hierarchy: Category, Subcategory")

    return unified_data

# Create unified dataset
unified_data = create_unified_dataset(hierarchy_data, marketing_daily, nps_daily)

# Display sample
print("\nSample of Unified Dataset:")
display(unified_data.head())

# Save the processed data
unified_data.to_csv('data/processed/unified_mmm_data.csv', index=False)
print("\n[OK] Saved unified dataset to 'data/processed/unified_mmm_data.csv'")

## Step 6: Data Validation and Summary Statistics

Let's validate our unified dataset and understand its characteristics:

In [ ]:
def validate_unified_dataset(unified_data):
    '''
    Comprehensive validation of the unified dataset.
    Ensures data is ready for modeling.
    '''

    print("UNIFIED DATASET VALIDATION")
    print("=" * 60)

    # Basic statistics
    print("\n1. DATASET OVERVIEW:")
    print("-" * 40)
    print(f"Total records: {len(unified_data):,}")
    print(f"Date range: {unified_data['Date'].min()} to {unified_data['Date'].max()}")
    print(f"Days covered: {unified_data['Date'].nunique()}")
    print(f"Categories: {unified_data['product_category'].nunique()}")
    print(f"Subcategories: {unified_data['product_subcategory'].nunique()}")

    # Sales statistics
    print("\n2. SALES METRICS:")
    print("-" * 40)
    print(f"Total GMV: ${unified_data['GMV'].sum():,.0f}")
    print(f"Average daily GMV: ${unified_data.groupby('Date')['GMV'].sum().mean():,.0f}")
    print(f"Total units sold: {unified_data['Units'].sum():,.0f}")

    # Marketing statistics
    print("\n3. MARKETING INVESTMENT:")
    print("-" * 40)
    total_marketing = unified_data[marketing_channels].sum().sum()
    print(f"Total marketing spend: ${total_marketing:,.0f}")

    for channel in marketing_channels[:5]:  # Top 5 channels
        if channel in unified_data.columns:
            spend = unified_data[channel].sum()
            pct = (spend / total_marketing) * 100
            print(f"  {channel}: ${spend:,.0f} ({pct:.1f}%)")

    # Data completeness
    print("\n4. DATA COMPLETENESS:")
    print("-" * 40)
    missing_pct = (unified_data.isnull().sum() / len(unified_data)) * 100
    complete_features = (missing_pct == 0).sum()
    print(f"Features with no missing values: {complete_features}/{len(missing_pct)}")

    if missing_pct.sum() > 0:
        print("\nFeatures with missing values:")
        print(missing_pct[missing_pct > 0])

    return True

# Validate the dataset
is_valid = validate_unified_dataset(unified_data)

if is_valid:
    print("\n" + "=" * 60)
    print("[OK] DATASET READY FOR MODELING!")
    print("=" * 60)

## Key Takeaways

### What We've Accomplished:
1. **Loaded three diverse data sources** with different granularities
2. **Created hierarchical aggregation** to capture product-level patterns
3. **Aligned time frequencies** through intelligent interpolation
4. **Built a unified dataset** ready for feature engineering

### Why This Matters for MMM:
- **Hierarchical structure** allows us to model different product responses to marketing
- **Daily granularity** captures immediate and lagged marketing effects
- **Unified dataset** ensures consistent analysis across all components

### Data Characteristics for Modeling:
- **Response variable (Y):** GMV at category/subcategory level
- **Marketing predictors (X):** 8+ marketing channels with daily spend
- **Control variables:** Price, discount, NPS, seasonality
- **Hierarchy:** Category and subcategory for pooled learning

### Next Steps:
In the next notebook, we'll engineer advanced features including:
- Adstock transformations for marketing carryover
- Beta-Gamma features for saturation curves
- Price elasticity features
- Seasonal decomposition

---

**Remember:** The quality of your data foundation determines the ceiling of your model's performance. We've built a solid foundation that preserves important patterns while maintaining data integrity.